# Tutorial 1: Matching

## 0. Import libraries 

In [1]:
import warnings 
from hypex import Matcher
from hypex.dataset.dataset import Dataset

warnings.simplefilter(action='ignore', category=FutureWarning)

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Create or upload your dataset  
In this case we will create random dataset with known effect size  
If you have your own dataset, go to the part 2 


In [2]:
data = Dataset(na_columns=['feature_col_3', 'feature_col_2'], 
               num_main_causes_cols=2)
data.df

,info_col_1,info_col_2,feature_col_1,feature_col_2,feature_col_3,feature_col_4,treatment,outcome
0,4396,W,female,Credit,NaN,2.0,0.0,1.566020
1,10600,W,female,NaN,-0.108763,3.0,1.0,7.407258
2,7642,W,female,Deposit,-0.111189,3.0,0.0,4.501391
3,2491,W,female,Deposit,-1.083582,0.0,1.0,-1.238530
4,1129,D,male,Investment,0.234198,2.0,1.0,4.486653
...,...,...,...,...,...,...,...,...
4995,14146,D,male,Investment,-0.826419,2.0,0.0,3.204788
4996,13834,W,female,Credit,-1.006133,2.0,1.0,3.670445
4997,4321,D,female,Credit,0.401821,2.0,1.0,6.346304
4998,5902,W,female,Deposit,1.170462,1.0,0.0,2.708312


In [4]:
data.df.columns

Index(['info_col_1', 'info_col_2', 'feature_col_1', 'feature_col_2',
       'feature_col_3', 'feature_col_4', 'treatment', 'outcome'],
      dtype='object')

In [6]:
data.df.treatment.value_counts()

treatment
1.0    2509
0.0    2491
Name: count, dtype: int64

In [7]:
data.df.isna().sum()

info_col_1         0
info_col_2         0
feature_col_1      0
feature_col_2    500
feature_col_3    500
feature_col_4      0
treatment          0
outcome            0
dtype: int64

## 2. Matching  
### 2.0 Init params
info_col used to define informative attributes that should not be part of matching, such as user_id  
But to explicitly store this column in the table, so that you can compare directly after computation

In [18]:
info_col = [data.info_col_names[0]]

outcome = data.outcome_name[0]
treatment = data.treatment_name[0]

### 2.1 Simple matching
This is the easiest way to initialize and calculate metrics on a Matching task  
Use it when you are clear about each attribute or if you don't have any additional task conditions (Strict equality for certain features) 

In [19]:
# Standard model with base parameters
model = Matcher(input_data=data.df, outcome=outcome, treatment=treatment, info_col=info_col)
results, quality_results, df_matched = model.estimate()

[23.11.2023 14:11:19 | hypex | INFO]: Number of NaN values filled with zeros: 1000
Get treated index: 100%|██████████| 5000/5000 [00:00<00:00, 8958.10it/s]  


In [20]:
results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,2.523897,0.070661,0.0,2.385402,2.662392,outcome
ATC,2.512587,0.080871,0.0,2.354081,2.671093,outcome
ATT,2.535126,0.077464,0.0,2.383296,2.686956,outcome


In [21]:
quality_results.keys()

dict_keys(['psi', 'ks_test', 'smd', 'repeats'])

In [22]:
quality_results['ks_test']

,match_control_to_treat,match_treat_to_control
feature_col_3,0.835224,0.957223
feature_col_4,1.000000,1.000000


In [23]:
df_matched

,index,feature_col_3,feature_col_4,info_col_2_W,feature_col_1_male,feature_col_2_Credit,feature_col_2_Deposit,feature_col_2_Investment,feature_col_3_matched,feature_col_4_matched,...,feature_col_1_male_matched,feature_col_2_Credit_matched,feature_col_2_Deposit_matched,feature_col_2_Investment_matched,index_matched,outcome,outcome_matched,outcome_matched_bias,treatment,treatment_matched
0,10600,-0.108763,3.0,1,0,0,0,0,0.302256,3.0,...,0.0,0.0,0.0,0.0,[4201],7.407258,2.503504,5.289862,1,0
1,2491,-1.083582,0.0,1,0,0,1,0,-1.089027,0.0,...,0.0,0.0,1.0,0.0,[7849],-1.238530,-0.656974,-0.586672,1,0
2,1129,0.234198,2.0,0,1,0,0,1,0.218125,2.0,...,1.0,0.0,0.0,1.0,[14836],4.486653,3.271825,1.199730,1,0
3,1843,1.184712,0.0,0,0,0,1,0,1.128866,0.0,...,0.0,0.0,1.0,0.0,[5635],2.490608,0.902258,1.535890,1,0
4,8302,-0.399125,0.0,1,0,0,1,0,-0.418014,0.0,...,0.0,0.0,1.0,0.0,[13174],-0.601565,2.201483,-2.820793,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2486,11128,-0.334717,1.0,1,0,1,0,0,-0.360573,1.0,...,0.0,1.0,0.0,0.0,[4171],0.835190,2.867754,2.084634,0,1
2487,13699,-0.374379,2.0,0,1,1,0,0,-0.373718,2.0,...,1.0,1.0,0.0,0.0,[4819],-0.192835,3.867719,4.059224,0,1
2488,14146,-0.826419,2.0,0,1,0,0,1,-0.850079,2.0,...,1.0,0.0,0.0,1.0,[14371],3.204788,4.830451,1.673309,0,1
2489,5902,1.170462,1.0,1,0,0,1,0,1.199764,1.0,...,0.0,0.0,1.0,0.0,[7618],2.708312,3.830416,1.063096,0,1


In [25]:
df_matched[df_matched['info_col_2_W'] != df_matched['info_col_2_W_matched']]

,index,feature_col_3,feature_col_4,info_col_2_W,feature_col_1_male,feature_col_2_Credit,feature_col_2_Deposit,feature_col_2_Investment,feature_col_3_matched,feature_col_4_matched,...,feature_col_1_male_matched,feature_col_2_Credit_matched,feature_col_2_Deposit_matched,feature_col_2_Investment_matched,index_matched,outcome,outcome_matched,outcome_matched_bias,treatment,treatment_matched


### 2.2 Matching with a fixed variable  
Used when you have categorical feature(s) that you want to compare by strict equality  
group_col is used for strict comparison of categorical features.  
In our case there is only one attribute  
If there are several such attributes, you should make one of them and use it

In [26]:
group_col = 'info_col_2'

In [27]:
model = Matcher(input_data=data.df, outcome=outcome, treatment=treatment,
                info_col=info_col, group_col=group_col)
results, quality_results, df_matched = model.estimate()

[23.11.2023 14:12:12 | hypex | INFO]: Number of NaN values filled with zeros: 1000
Get treated index by group W: 100%|██████████| 4/4 [00:00<00:00, 29.58it/s]  


In [28]:
results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,2.523897,0.070661,0.0,2.385402,2.662392,outcome
ATC,2.512588,0.080870,0.0,2.354082,2.671094,outcome
ATT,2.535126,0.077464,0.0,2.383296,2.686955,outcome


In [29]:
df_matched

,index,feature_col_3,feature_col_4,feature_col_1_male,feature_col_2_Credit,feature_col_2_Deposit,feature_col_2_Investment,info_col_2,feature_col_3_matched,feature_col_4_matched,...,feature_col_2_Credit_matched,feature_col_2_Deposit_matched,feature_col_2_Investment_matched,info_col_2_matched,index_matched,outcome,outcome_matched,outcome_matched_bias,treatment,treatment_matched
0,1129,0.234198,2.0,1,0,0,1,D,0.218125,2.0,...,0.0,0.0,1.0,D,[14836],4.486653,3.271825,1.199727,1,0
1,1843,1.184712,0.0,0,0,1,0,D,1.128866,0.0,...,0.0,1.0,0.0,D,[5635],2.490608,0.902258,1.535882,1,0
2,12010,-0.257334,3.0,1,0,0,0,D,-0.473090,3.0,...,0.0,0.0,0.0,D,[3043],6.411878,1.108409,5.100759,1,0
3,9295,-1.854170,0.0,1,0,1,0,D,-1.891715,0.0,...,0.0,1.0,0.0,D,[3379],-0.969977,-0.892780,-0.112472,1,0
4,2455,-0.372677,3.0,1,0,1,0,D,-0.371943,3.0,...,0.0,1.0,0.0,D,[445],7.222956,1.267117,5.956529,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2486,253,-1.457601,1.0,1,0,1,0,W,-1.441001,1.0,...,0.0,1.0,0.0,W,[49],-0.190917,0.193336,0.350827,0,1
2487,3814,-0.858134,3.0,1,0,1,0,W,-0.879061,3.0,...,0.0,1.0,0.0,W,[8989],1.941070,7.046109,5.147178,0,1
2488,11128,-0.334717,1.0,0,1,0,0,W,-0.360573,1.0,...,1.0,0.0,0.0,W,[4171],0.835190,2.867754,2.084630,0,1
2489,5902,1.170462,1.0,0,0,1,0,W,1.199764,1.0,...,0.0,1.0,0.0,W,[7618],2.708312,3.830416,1.063101,0,1


In [31]:
df_matched[df_matched['info_col_2'] != df_matched['info_col_2_matched']]

,index,feature_col_3,feature_col_4,feature_col_1_male,feature_col_2_Credit,feature_col_2_Deposit,feature_col_2_Investment,info_col_2,feature_col_3_matched,feature_col_4_matched,...,feature_col_2_Credit_matched,feature_col_2_Deposit_matched,feature_col_2_Investment_matched,info_col_2_matched,index_matched,outcome,outcome_matched,outcome_matched_bias,treatment,treatment_matched


## 3. Results  
### 3.1 ATE, ATT, ATC

In [32]:
results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,2.523897,0.070661,0.0,2.385402,2.662392,outcome
ATC,2.512588,0.080870,0.0,2.354082,2.671094,outcome
ATT,2.535126,0.077464,0.0,2.383296,2.686955,outcome


### 3.2 SMD, PSI, KS-test, repeats

In [33]:
quality_results.keys()

dict_keys(['psi', 'ks_test', 'smd', 'repeats'])

In [34]:
quality_results['psi']

,column_treated,anomaly_score_treated,check_result_treated,column_untreated,anomaly_score_untreated,check_result_untreated
0,feature_col_1_male_treated,0.0,OK,feature_col_1_male_untreated,0.00,OK
1,feature_col_2_Credit_treated,0.0,OK,feature_col_2_Credit_untreated,0.00,OK
2,feature_col_2_Deposit_treated,0.0,OK,feature_col_2_Deposit_untreated,0.00,OK
3,feature_col_2_Investment_treated,0.0,OK,feature_col_2_Investment_untreated,0.00,OK
4,feature_col_3_treated,0.0,OK,feature_col_3_untreated,0.01,OK
5,feature_col_4_treated,0.0,OK,feature_col_4_untreated,0.00,OK
6,info_col_2_treated,0.0,OK,info_col_2_untreated,0.00,OK


In [35]:
quality_results['ks_test']

,match_control_to_treat,match_treat_to_control
feature_col_3,0.835224,0.957223
feature_col_4,1.000000,1.000000


In [36]:
quality_results['repeats']

{'match_control_to_treat': 0.39, 'match_treat_to_control': 0.39}

### 3.3 Validation
Validates estimated effect:
1. by replacing real treatment (`random_treatment`) with random placebo treatment.
 Estimated effect must be droped to zero;
2. by adding random feature (`random_feature`). Estimated effect shouldn't change
significantly, p-val < 0.05;
3. estimates effect on subset of data (`subset_refuter`) (default fraction is 0.8). Estimated effect
shouldn't change significantly, p-val < 0.05.

In [37]:
model.validate_result(refuter="random_treatment", effect_type="att", n_sim=10)

100%|██████████| 10/10 [00:11<00:00,  1.12s/it]


{'outcome': [0.004513215149313391, 0.0]}

## 4. Save model

In [38]:
model.save("test_model.pickle")

In [39]:
model2 = Matcher.load("test_model.pickle")

In [40]:
model2.results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,2.523897,0.070661,0.0,2.385402,2.662392,outcome
ATC,2.512588,0.080870,0.0,2.354082,2.671094,outcome
ATT,2.535126,0.077464,0.0,2.383296,2.686955,outcome


In [41]:
model.results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,2.523897,0.070661,0.0,2.385402,2.662392,outcome
ATC,2.512588,0.080870,0.0,2.354082,2.671094,outcome
ATT,2.535126,0.077464,0.0,2.383296,2.686955,outcome
